In [1]:
import visual_behavior_glm.GLM_params as glm_params
import visual_behavior_glm.GLM_analysis_tools as gat
import visual_behavior_glm.GLM_visualization_tools as gvt
from visual_behavior_glm.glm import GLM
import matplotlib.pyplot as plt
import visual_behavior.data_access.loading as loading
import visual_behavior.database as db
import plotly.express as px

import pandas as pd
import numpy as np
import os

import seaborn as sns

import visual_behavior.plotting as vbp

import plotly.graph_objects as go
import itertools

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
%widescreen

applied a custom magic command to make full use of screen width
will only work if command is defined locally
replace with the following to replicate functionality: 
	from IPython.core.display import display, HTML
	display(HTML("<style>.container { width:100% !important; }</style>")


In [3]:
glm_version = '9d_L2_optimize_by_session'
dropout_summary = gat.retrieve_results(search_dict = {'glm_version': glm_version}, results_type='summary')
dropout_summary.sample(5)

,_id,dropout,absolute_change_from_full,variance_explained,variance_explained_full,fraction_change_from_full,cell_specimen_id,adj_fraction_change_from_full,adj_variance_explained,adj_variance_explained_full,glm_version,ophys_experiment_id,ophys_session_id,entry_time_utc,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,has_events,session_number
5033446,5f9a1d34573381f0278cf74f,task,-0.001671,0.038445,0.040117,-0.041663,962626671,-0.087466,0.062373,0.068351,9d_L2_optimize_by_session,792813858,792327341,2020-10-31 11:55:30.474336,792477679,814796612,VisualBehavior,container_qc,passed,20181210_412366_1ImagesA,OPHYS_1_images_A,CAM2P.4,2018-12-10 16:34:08.000000,765187424,756674785,F,147.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,756674785,Slc17a7-IRES2-Cre,NaN,validate_one_stimulus_per_aborted_trial,0,True,Slc17a7_VISp_375,False,1
1438808,5f997fc05e894a1092a713f9,time,-0.000039,0.008515,0.008554,-0.004580,1028758329,-0.004580,0.008515,0.008554,9d_L2_optimize_by_session,887386955,887031077,2020-10-31 02:08:59.363817,887165432,1018027543,VisualBehaviorMultiscope,container_qc,passed,20190614_451787_6imagesB,OPHYS_6_images_B,MESO.1,2019-06-14 12:31:30.000000,846136902,837581585,M,135.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",77,VISp,NaN,837581585,Slc17a7-IRES2-Cre,NaN,NaN,0,True,Slc17a7_VISp_77,False,6
2553958,5f99a81acacbb2e06abfb3da,single-image5,-0.067730,0.006595,0.074325,-0.088732,1028825385,-0.917216,0.067490,0.073581,9d_L2_optimize_by_session,974358973,973701907,2020-10-31 04:00:08.300423,973744975,1018027828,VisualBehaviorMultiscope,container_qc,passed,20191029_484627_Ophys4,OPHYS_4_images_B,MESO.1,2019-10-29 10:41:35.836194,943395690,930825455,M,113.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",261,VISp,NaN,930825455,Slc17a7-IRES2-Cre,NaN,NaN,0,True,Slc17a7_VISp_261,False,4
2664207,5f99a9cc4b2d041f6f763a11,single-face_motion_PC_3,-0.007512,0.000102,0.007614,0.000000,1028795758,0.000000,0.000102,0.007609,9d_L2_optimize_by_session,903485705,902884228,2020-10-31 03:53:14.731037,902995627,1018027650,VisualBehaviorMultiscope,container_qc,passed,20190710_456915_Ophys1,OPHYS_1_images_A,MESO.1,2019-07-10 12:09:00.013603,852260338,843122504,F,139.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",275,VISp,NaN,843122504,Slc17a7-IRES2-Cre,NaN,z_drift_corr_um_diff,0,True,Slc17a7_VISp_275,False,1
2872250,5f99b0d80a16cbe04230cd25,single-pre_licks,-0.017084,0.000000,0.017084,0.000000,962617091,0.000000,0.000000,0.000000,9d_L2_optimize_by_session,862848066,860331718,2020-10-28 17:56:40.232860,860533862,845588020,VisualBehavior,container_qc,passed,20190501_445270_4imagesB,OPHYS_4_images_B,CAM2P.3,2019-05-01 17:05:37.000000,828775962,820871408,M,118.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,820871408,Slc17a7-IRES2-Cre,NaN,NaN,2,True,Slc17a7_VISp_375,True,4


In [4]:
def get_dominant_dropout(row, cols_to_check):
    return row[cols_to_check].astype(float).idxmin()

def map_session_types(session_type):
    session_id = int(session_type[6:7])
    return session_id

def make_categorical(df, column):
    df['{}_categorical'.format(column)] = pd.Categorical(df[column], ordered=True).codes

def get_dominant_dropout_summary(dropout_summary=None, glm_version=None):

    if dropout_summary is None:
        assert glm_version is not None, 'must pass glm_version if not passing a dropout summary'
        dropout_summary = gat.retrieve_results(search_dict = {'glm_version': glm_version}, results_type='summary')

    all_dropouts = np.sort([dropout for dropout in dropout_summary['dropout'].unique() if not dropout.startswith('single')])
    # get pivoted dropout summary
    # one row per session/cell, columns with scores for each dropout and
    #  - dominant dropout (string and categorical)
    #  - session id (int and categorical)
    dropout_summary_pivoted = gat.build_pivoted_results_summary(results_summary=dropout_summary, cutoff=0.01, value_to_use='adj_fraction_change_from_full')

    cols_to_check = [col for col in all_dropouts if not col.startswith('image') and col!='visual']
    dropout_summary_pivoted['dominant_dropout'] = dropout_summary_pivoted.apply(get_dominant_dropout, axis=1, args=[cols_to_check])

    dropout_summary_pivoted['session_id'] = dropout_summary_pivoted['session_type'].map(lambda st:map_session_types(st))

    for column in ['cre_line','equipment_name','targeted_structure','session_id','dominant_dropout']:
        make_categorical(dropout_summary_pivoted, column)

    dominant_dropout_summary = (
        dropout_summary_pivoted
        .drop_duplicates(['cell_specimen_id','session_id'])
        .pivot(index='cell_specimen_id',columns='session_id',values='dominant_dropout_categorical')
        .rename(columns={k:'session_{}'.format(k) for k in dropout_summary_pivoted['session_id'].unique()})
    )

    cols_to_merge = [
        'cell_specimen_id',
        'cre_line',
        'cre_line_categorical',
        'imaging_depth',
        'targeted_structure',
        'targeted_structure_categorical',
        'equipment_name',
        'equipment_name_categorical',
    ]

    dominant_dropout_summary = dominant_dropout_summary.merge(
        dropout_summary_pivoted[cols_to_merge],
        left_on='cell_specimen_id',
        right_on='cell_specimen_id',
        how='left',
    ).set_index('cell_specimen_id')
    
    categories = list(np.sort([c for c in dropout_summary_pivoted['dominant_dropout'].unique() if pd.notnull(c)]))
    
    return categories, dominant_dropout_summary

In [5]:
categories, dominant_dropout_summary = get_dominant_dropout_summary(dropout_summary)

In [6]:
def sankey_diagram(data_to_plot, categories, title_text=''):
    # define colormap and categories
    categories = list(np.sort([c for c in dropout_summary_pivoted['dominant_dropout'].unique() if pd.notnull(c)]))
    cmap = ['rgb({}, {}, {})'.format(int(255*v[0]),int(255*v[1]),int(255*v[2])) for v in sns.color_palette("Dark2", len(categories))]
    sessions = np.arange(1,7)

    # initialize the link dictionary
    link_dict = dict(
        source = [], # indices correspond to labels, eg A1, A2, A2, B1, ...
        target = [],
        value =  [],
        label =  []

    )

    # define values and session IDs
    possible_vals = np.arange(len(categories))
    session_pairs = [(1,2),(2,3),(3,4),(4,5),(5,6)]
    # iterate over session pairs:
    for ii, (session_id_1, session_id_2) in enumerate(session_pairs):
        # iterate over all possible combination of dropout values:
        for first_val, second_val in list(itertools.product(possible_vals, possible_vals)):
            # find all rows that meet the condition of having the two sessions equal to the two values
            query = 'session_{} == {} and session_{} == {}'.format(session_id_1, first_val, session_id_2, second_val)
            # update the link dict with relevant data
            link_dict['source'].append(first_val + len(categories)*ii)
            link_dict['target'].append(second_val+ len(categories)*(ii+1))
            link_dict['value'].append(len(data_to_plot.query(query)))

            # calculate the fraction of the total, update the label
            label_total = len(data_to_plot.query(query.split(' and')[0]))
            label_fraction = 0 if label_total == 0 else len(data_to_plot.query(query))/label_total
            label = '{} to {} probability: {:0.3f}'.format(categories[first_val], categories[second_val], label_fraction)
            link_dict['label'].append(label)

    # add category labels
    category_labels = []
    for session_id in sessions:
        for category_index, category in enumerate(categories):
            query = 'session_{} == {}'.format(session_id, category_index)
            fraction = len(data_to_plot.query(query))/len(data_to_plot)
            category_labels.append('{}\n({:0.1f}%)'.format(category, 100*fraction))
    #         print('index = {}\nnumber = {}\n{}\n({:0.1f}%)'.format(category_index, len(data_to_plot.query(query)), category, 100*fraction))

    # define the sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = len(sessions)*category_labels,
          color = len(sessions)*cmap
        ),
        link = link_dict)])

    # update the layout
    fig.update_layout(title_text=title_text, font_size=10)

        # return the figure object:
    return fig

In [7]:
sort_by = [
    'cre_line_categorical',
    'session_1',
    'session_2',
    'session_3',
    'session_4',
    'session_5',
    'session_6',
    'imaging_depth'
]
cre_lines = np.sort(dropout_summary.cre_line.unique())


for cre_line in cre_lines:
    data_to_plot = dominant_dropout_summary.sort_values(by=sort_by).dropna().query('cre_line == "{}"'.format(cre_line))
    fig = sankey_diagram(data_to_plot, categories, title_text='Sankey plot, GLM verion = {}, cre-line = {}'.format(glm_version, cre_line))
    fig.write_html("/home/dougo/code/dougollerenshaw.github.io/figures_to_share/sankey_diagram_glm_version_{}_cre_line_{}.html".format(glm_version, cre_line))
    fig.show()

NameError: name 'dropout_summary_pivoted' is not defined